In [1]:
pip install numpy==1.24.3 pandas==1.5.3 scikit-learn==1.2.2

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 6.6 MB/s eta 0:00:00m eta 0:00:010:01:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.1/12.1 MB 10.0 MB/s eta 0:00:00m eta 0:00:010:01:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.6/9.6 MB 10.8 MB/s eta 0:00:00m eta 0:00:010:01:01
  Attempting uninstall: numpy
    Found existing installation: numpy 2.2.4
    Uninstalling numpy-2.2.4:
      Successfully uninstalled numpy-2.2.4
  Attempting uninstall: pandas
    Found existing installation: pandas 2.2.3
    Uninstalling pandas-2.2.3:
      Successfully uninstalled pandas-2.2.3
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.7.0
    Uninstalling scikit-learn-1.7.0:
      Successfully uninstalled scikit-learn-1.7.0

[notice] A new release of pip available: 22.3.1 -> 25.1.1
[notice] To update, run: pip3.10 install --upgrade pip
Note: you may need to restart th

In [2]:
import pandas as pd
from sklearn.ensemble import IsolationForest
from sklearn.preprocessing import StandardScaler

In [7]:
df = pd.read_csv("noisy_isolation_features_dataset.csv")
# df.sample(10)

In [9]:
df.dtypes

attempts_in_60s        int64
unique_users_in_60s    int64
invalid_user           int64
success_after_fail     int64
true_user              int64
anomaly                int64
dtype: object

In [5]:
X = df.drop(['anomaly'],axis = 1)

In [10]:
print(type(X))
print(X.columns)


<class 'pandas.core.frame.DataFrame'>
Index(['attempts_in_60s', 'unique_users_in_60s', 'invalid_user',
       'success_after_fail', 'true_user'],
      dtype='object')


In [6]:
iso_model = IsolationForest(
    n_estimators=50,
    contamination=0.05,  # or even 0.1 for more aggressive detection
    max_samples=0.5,
    random_state=42
)

iso_model.fit(X)

/home/zero/.local/lib/python3.10/site-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but IsolationForest was fitted with feature names
  warnings.warn(


IsolationForest(contamination=0.05, max_samples=0.5, n_estimators=50,
                random_state=42)

In [11]:
# === Predict anomalies ===
# -1 = anomaly, 1 = normal
features_df = X
features_df['anomalyy'] = iso_model.predict(X)

# === See results ===
# print(features_df[['qid', 'anomaly']].sample(20))

In [12]:
features_df.anomalyy.value_counts()

 1    1901
-1      99
Name: anomalyy, dtype: int64

In [13]:
features_df[features_df['anomalyy']==-1]

,attempts_in_60s,unique_users_in_60s,invalid_user,success_after_fail,true_user,anomalyy
26,7,1,1,0,0,-1
72,4,1,0,1,0,-1
153,4,2,0,0,0,-1
155,27,2,1,0,0,-1
157,8,1,1,1,0,-1
...,...,...,...,...,...,...
1910,8,1,0,0,0,-1
1922,4,2,0,1,0,-1
1925,6,2,1,1,0,-1
1969,30,6,1,0,0,-1


In [14]:
features_df.sample(10)

,attempts_in_60s,unique_users_in_60s,invalid_user,success_after_fail,true_user,anomalyy
234,6,2,0,1,0,1
1905,6,1,0,1,0,1
1726,6,2,0,0,0,1
978,7,1,0,0,1,1
1984,6,2,0,1,0,1
1867,13,3,1,0,0,1
1038,9,2,0,1,0,-1
1466,6,2,0,1,0,1
1157,6,2,0,1,0,1
1992,7,2,0,1,0,1


In [15]:
import joblib
joblib.dump(iso_model, 'bruteforce_model.pkl')

['bruteforce_model.pkl']